In [1]:
from sheets import Sheets

sh = Sheets('creds/lifts-service.json')
sh.auth()
sheet = sh.get_sheet_by_key('1XmPko5pbcw0HdvPZeL7Fw8tfX4zhvxREUDGISC54jwk')
ws = sh.get_0th_worksheet(sheet)  
%load_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import altair as alt
import chart_generator as cg

%autoreload 2

alt.renderers.enable('mimetype')

lr = cg.ChartGeneratorLeftRight(ws)
single = cg.ChartGeneratorSingle(ws)
charter = cg.ChartGenerator(ws)

x = charter.chart_style()






In [12]:
# weight graph (lower is better)

chart = single.make_chart(4, False)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [4]:
# Hips
chart = single.make_chart(9)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [5]:
# Waist (lower is better)
chart = single.make_chart_with_hover_tool_tip(10)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [6]:
# Chest

chart = single.make_chart_with_tool_tip(11)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [7]:
# Calve Graph
chart = lr.make_chart_hover(5, 6)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [8]:
# Thighs
chart = lr.make_chart_with_hover_tool_tip(7,8)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [9]:
# Forearms
chart = lr.make_chart(12,13)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [10]:
# Biceps
chart = lr.make_chart_with_hover_tool_tip(14, 15)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html
